In [1]:
!pip install python-dotenv
!pip install langchain
!pip install langchain-openai
!pip install langchain-chroma
!pip install wikipedia
!pip install pypdf
!pip install pinecone
!pip install faiss-cpu
!pip install lark
!pip install rank_bm25
!pip install langchain_community
!pip install langchain_experimental
!pip install langchainhub

In [141]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
from dotenv import load_dotenv, find_dotenv

In [4]:
load_dotenv("./.env") #load_dotenv(find_dotenv(), everride = True)
# Se cargan las credenciales

dotenv_path = "/content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/.env"
load_dotenv(dotenv_path)
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI()
output = llm.invoke("Cual es la capital de Argentina") # model = "gpt-3.5-turbo"
print(output.content)

La capital de Argentina es Buenos Aires.


# 1- AGENTES: Las bases

## A- Definimos las TOOLS

### A.1- Tool número 1: TAVILY

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults # para poder hacer busquedas de urls

In [8]:
search = TavilySearchResults()

In [9]:
search.name

'tavily_search_results_json'

In [10]:
search.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [11]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1721777606, 'localtime': '2024-07-23 16:33'}, 'current': {'last_updated_epoch': 1721777400, 'last_updated': '2024-07-23 16:30', 'temp_c': 20.6, 'temp_f': 69.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 259, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 57, 'cloud': 12, 'feelslike_c': 20.6, 'feelslike_f': 69.0, 'windchill_c': 20.6, 'windchill_f': 69.0, 'heatindex_c': 21.3, 'heatindex_f': 70.3, 'dewpoint_c': 11.9, 'dewpoint_f': 53.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 18.0, 'gust_kph': 28.9}}"},
 {'url': 'https://www.timeanddate.com/weather/usa/sa

### A.2- Tool número 2: RETRIEVER

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [13]:
retriever.invoke("how to upload a dataset?")[0].page_content

'description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";const client = new'

In [14]:
from langchain.tools.retriever import create_retriever_tool

In [15]:
retriever_tool = create_retriever_tool(
    retriever = retriever,
    name  = "langsmith_search",
    description  = "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### A.3- Creamos la lista de Tools

In [16]:
tools = [search, retriever_tool]

## B- Definimos el LLM

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature = 0)

In [18]:
  from langchain import hub

  # Get the prompt to use - you can modify this!
  prompt = hub.pull("hwchase17/openai-functions-agent")


**Inspeccionemos un poco el prompt importado:**

In [19]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplat

In [20]:
prompt.input_variables

['agent_scratchpad', 'input']

In [21]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

**Veamos otro ejemplo:**

In [22]:
prompt2 = hub.pull("hwchase17/react")

In [23]:
prompt2

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [24]:
prompt2.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

In [25]:
prompt2.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

## C- Definimos el agente

In [26]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

## D- Usamos el agente

In [27]:
from langchain.agents import AgentExecutor

In [28]:
from langchain.chains import LLMChain
conversation = LLMChain(
    llm = llm,
    prompt = prompt,
    verbose = True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [29]:
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True, conversation=conversation)

In [30]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [31]:
result = agent_executor.invoke({"input": "how can langsmith help with testing?"})
result["output"]



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPEN

'LangSmith is a platform for building production-grade LLM applications that can help with testing. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. Here are some ways LangSmith can assist with testing:\n\n1. **Installation**: You can install LangSmith using Python or TypeScript by running the following commands:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. **Create an API Key**: Generate an API key by heading to the Settings page and clicking on "Create API Key."\n\n3. **Set up your environment**: Configure your environment by setting the necessary environment variables like `LANGCHAIN_TRACING_V2` and `LANGCHAIN_API_KEY`.\n\n4. **Log your first trace**: LangSmith provides multiple ways to log traces for testing purposes. You can log your first trace to start evaluating your application.\n\n5. **Run evaluations**: LangSmith allows you to run evaluations on your test cases

In [32]:
result = agent_executor.invoke({"input": "whats the weather in sf?"})
result["output"]



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1721777606, 'localtime': '2024-07-23 16:33'}, 'current': {'last_updated_epoch': 1721777400, 'last_updated': '2024-07-23 16:30', 'temp_c': 20.6, 'temp_f': 69.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 259, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 57, 'cloud': 12, 'feelslike_c': 20.6, 'feelslike_f': 69.0, 'windchill_c': 20.6, 'windchill_f': 69.0, 'heatindex_c': 21.3, 'heatindex_f': 70.3, 'dewpoint_c': 11.9, 'dewpoint_f': 53.4, 'vis_km': 10

'The current weather in San Francisco is sunny with a temperature of 69.0°F (20.6°C). The wind speed is 17.6 kph coming from the west. The humidity is at 57%, and there is no precipitation at the moment.'

**El agente tiene memoria?**

In [33]:
result = agent_executor.invoke({"input": "What was my first question?"})
result["output"]



> Entering new AgentExecutor chain...
I don't have the ability to remember your previous questions. How can I assist you today?

> Finished chain.


"I don't have the ability to remember your previous questions. How can I assist you today?"

## E- Memoria

**Agregando memoria, primer ejemplo:**

In [34]:
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model = "gpt-3.5-turbo-1106", temperature = 0)
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent = agent,
    verbose = True,
    tools = tools,
    memory = memory
)

In [35]:
result = agent_executor.invoke({"input": "My name is Javier"})
result["output"]



> Entering new AgentExecutor chain...
Hello Javier! How can I assist you today?

> Finished chain.


'Hello Javier! How can I assist you today?'

In [36]:
result = agent_executor.invoke({"input": "What is my name? Don't look the tools!"})
result["output"]



> Entering new AgentExecutor chain...
Your name is Javier.

> Finished chain.


'Your name is Javier.'

**Agregando memoria, segundo ejemplo:**

In [37]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_tool_calling_agent, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
load_dotenv()

chat = ChatOpenAI()

prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(content=(
            "Eres una IA muy amable, solamente responde las preguntas del usuario."
        )),
        MessagesPlaceholder(variable_name = "chat_history"),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name = "agent_scratchpad")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
tools = [search]
#create_tool_calling_agent
#create_openai_tools_agent
agent = create_openai_tools_agent(
    llm=chat,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools,
    memory=memory
)

In [38]:
for i in prompt:
    print(i)
    print()

('name', None)

('input_variables', ['agent_scratchpad', 'chat_history', 'input'])

('optional_variables', [])

('input_types', {'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]})

('output_parser', None)

('partial_variables', {})

('metadata', None)

('tags', None)

('messages', [SystemMessage(content='Eres una IA muy amable, solamente responde las preguntas del usuario.'), MessagesPlaceholder(variable_name='chat_history'), HumanM

In [39]:
rta1 = agent_executor.invoke(
    {"input": "Hello my name is Javier!!."}
)
rta1



> Entering new AgentExecutor chain...
Hello Javier! How can I assist you today?

> Finished chain.


{'input': 'Hello my name is Javier!!.',
 'chat_history': [HumanMessage(content='Hello my name is Javier!!.'),
  AIMessage(content='Hello Javier! How can I assist you today?')],
 'output': 'Hello Javier! How can I assist you today?'}

In [40]:
rta1["chat_history"]

[HumanMessage(content='Hello my name is Javier!!.'),
 AIMessage(content='Hello Javier! How can I assist you today?')]

In [41]:
rta1["output"]

'Hello Javier! How can I assist you today?'

In [42]:
rta2 = agent_executor.invoke(
    {"input": "What is my name?."}
)
rta2



> Entering new AgentExecutor chain...
Your name is Javier! How can I assist you, Javier?

> Finished chain.


{'input': 'What is my name?.',
 'chat_history': [HumanMessage(content='Hello my name is Javier!!.'),
  AIMessage(content='Hello Javier! How can I assist you today?'),
  HumanMessage(content='What is my name?.'),
  AIMessage(content='Your name is Javier! How can I assist you, Javier?')],
 'output': 'Your name is Javier! How can I assist you, Javier?'}

In [43]:
rta2["chat_history"]

[HumanMessage(content='Hello my name is Javier!!.'),
 AIMessage(content='Hello Javier! How can I assist you today?'),
 HumanMessage(content='What is my name?.'),
 AIMessage(content='Your name is Javier! How can I assist you, Javier?')]

In [44]:
rta2["output"]

'Your name is Javier! How can I assist you, Javier?'

In [45]:
rta3 = agent_executor.invoke(
    {"input": "What is 2 plus 6?."}
)
rta3["output"]



> Entering new AgentExecutor chain...
2 plus 6 equals 8. Is there anything else you would like to know or discuss, Javier?

> Finished chain.


'2 plus 6 equals 8. Is there anything else you would like to know or discuss, Javier?'

In [46]:
rta3["chat_history"]

[HumanMessage(content='Hello my name is Javier!!.'),
 AIMessage(content='Hello Javier! How can I assist you today?'),
 HumanMessage(content='What is my name?.'),
 AIMessage(content='Your name is Javier! How can I assist you, Javier?'),
 HumanMessage(content='What is 2 plus 6?.'),
 AIMessage(content='2 plus 6 equals 8. Is there anything else you would like to know or discuss, Javier?')]

In [47]:
rta4 = agent_executor.invoke(
    {"input": "And 2 more?"}
)
rta4["output"]



> Entering new AgentExecutor chain...
Adding 2 more to 8 gives you a total of 10. Is there anything else you would like to ask, Javier?

> Finished chain.


'Adding 2 more to 8 gives you a total of 10. Is there anything else you would like to ask, Javier?'

In [48]:
rta4["chat_history"]

[HumanMessage(content='Hello my name is Javier!!.'),
 AIMessage(content='Hello Javier! How can I assist you today?'),
 HumanMessage(content='What is my name?.'),
 AIMessage(content='Your name is Javier! How can I assist you, Javier?'),
 HumanMessage(content='What is 2 plus 6?.'),
 AIMessage(content='2 plus 6 equals 8. Is there anything else you would like to know or discuss, Javier?'),
 HumanMessage(content='And 2 more?'),
 AIMessage(content='Adding 2 more to 8 gives you a total of 10. Is there anything else you would like to ask, Javier?')]

In [49]:
rta4["output"]

'Adding 2 more to 8 gives you a total of 10. Is there anything else you would like to ask, Javier?'

# 2- AGENTES: ReAct

* Combina las capacidades de razonar (Re = Reasoning) con un prompt del estilo `chain of thoughts` y sus capacidades de actuar (Act = Acting) de los LLMs.
* Genera razonamientos y acciones específicas para cada uno en intervalos.
* Esto reduce las alucinaciones.

## A- No todo es perfecto

**Ejemplo matemática 1**

In [50]:
output = llm.invoke("What is the answer to 5.1 ** 7.3?")
print(output.content)

The answer to 5.1 ** 7.3 is approximately 10498.78.


In [51]:
5.1 ** 7.3

146306.05007233328

**Ejemplo matemática 2**

In [52]:
output = llm.invoke("What is the answer to 5.1 ** 1.7?")
print(output.content)

The answer to 5.1 ** 1.7 is approximately 11.89.


In [53]:
5.1 ** 1.7

15.953989407385711

**Ejemplo alucinación 1**

In [54]:
output = llm.invoke("Write and article about LangChain Agents")
print(output.content)

LangChain Agents: Revolutionizing the Language Services Industry

In today's globalized world, the need for language services has never been greater. From international business transactions to cross-cultural communication, the demand for professional language services is on the rise. However, the traditional language services industry has been plagued by inefficiencies, high costs, and a lack of transparency. This is where LangChain Agents comes in.

LangChain Agents is a revolutionary platform that is set to disrupt the language services industry. By leveraging blockchain technology and a decentralized network of language professionals, LangChain Agents is able to provide a more efficient, cost-effective, and transparent solution for language services.

One of the key features of LangChain Agents is its decentralized network of language professionals. Instead of relying on a centralized agency to provide language services, LangChain Agents connects clients directly with a global netw

**Ejemplo alucinación 2**

In [55]:
output = llm.invoke("What is Langchain?")
print(output.content)

Langchain is a blockchain-based platform that aims to provide a decentralized and secure solution for language translation and interpretation services. It utilizes smart contracts and a network of translators to facilitate seamless and efficient language services, while also ensuring transparency and trust in the translation process. The platform also offers a token-based incentive system to reward translators and encourage participation in the network.


## B- Implementemos una solucion

Puedo ir a buscar cómo se utiliza la Tool del REPL de Python

In [56]:
from langchain.agents import Tool
from langchain_experimental.tools import PythonREPLTool

In [57]:
python_repl_tool = PythonREPLTool()

In [58]:
python_repl_tool.run("print([n for n in range(1, 100) if n % 13 == 0])")

'[13, 26, 39, 52, 65, 78, 91]\n'

In [59]:
repl_tool = Tool.from_function(
    name = "python_repl",
    description = "A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func = python_repl_tool.run,
)

In [60]:
tools = [repl_tool]

In [61]:
prompt = hub.pull("hwchase17/react")
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [62]:
from langchain.agents import create_react_agent

In [63]:
agent = create_react_agent(llm, tools, prompt)

In [64]:
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [65]:
agent_executor.invoke({"input": "Calculate the square root of the factorial of 12 and display it."})



> Entering new AgentExecutor chain...
I should use the python_repl tool to calculate the square root of the factorial of 12 and display the result.
Action: python_repl
Action Input: print((12*11*10*9*8*7*6*5*4*3*2*1)**0.5)21886.105181141756
I now know the final answer
Final Answer: The square root of the factorial of 12 is approximately 21886.105181141756

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it.',
 'output': 'The square root of the factorial of 12 is approximately 21886.105181141756'}

In [66]:
import math
import numpy as np

resultado = math.factorial(12)
np.sqrt(resultado)

21886.105181141756

In [67]:
output = llm.invoke("What is the answer to 5.1 ** 7.3?") # model = "gpt-3.5-turbo"
print(output.content)

The answer to 5.1 ** 7.3 is approximately 10498.78.


In [68]:
5.1 ** 7.3

146306.05007233328

In [69]:
result = agent_executor.invoke({"input": "What is de answer to 5.1 ** 7.3"})
result



> Entering new AgentExecutor chain...
I should use the python_repl tool to calculate 5.1 ** 7.3
Action: python_repl
Action Input: 5.1 ** 7.3I should print out the result to see the answer
Action: python_repl
Action Input: print(5.1 ** 7.3)146306.05007233328
I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


{'input': 'What is de answer to 5.1 ** 7.3', 'output': '146306.05007233328'}

## C- Complejicemos un poco el caso

In [70]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

In [71]:
# Creamos el llm

llm = ChatOpenAI()

In [72]:
template = """"
Answer the following questions as best you can:
Questions: {q}
"""
prompt_template = PromptTemplate.from_template(template)

In [73]:
prompt = hub.pull("hwchase17/react")
print(f"prompt inputs: {prompt.input_variables}")
print()
print(f"prompt template: {prompt.template}") # para ver la chain of thoughts

prompt inputs: ['agent_scratchpad', 'input', 'tool_names', 'tools']

prompt template: Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [74]:
# Generamos la tool del REPL de Python

python_repl = PythonREPLTool()

repl_tool = Tool(
    name = "Python_repl",
    description = "A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func = python_repl.run,
)

In [75]:
# Generamos la tool de búsquedas en Wikipedia

api_wrapper = WikipediaAPIWrapper(top_k_results = 1, doc_content_chars_max = 100)
wikipedia = WikipediaQueryRun(api_wrapper = api_wrapper)

wikipedia_tool = Tool.from_function(
    name = "Wikipedia",
    description = "Useful when you need to look up a topic, or person on Wikipedia",
    func = wikipedia.run,
)

In [76]:
# Generamos una tool de un retriever

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever = retriever,
    name  = "langsmith_search",
    description  = "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [77]:
tools = [repl_tool, wikipedia_tool, retriever_tool]

In [78]:
agent = create_react_agent(llm, tools, prompt)

In [79]:
agent_executor = AgentExecutor(agent = agent,
                               tools = tools,
                               verbose = True,
                               handle_parsing_errors = True,
                               max_iterations = 4)

In [80]:
question = "Generate the first 20 numbers in the Fibonacci series"

In [81]:
result = agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...
I can use Python to generate the Fibonacci series.
Action: Python_repl
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    for i in range(2, n):
        fib_series.append(fib_series[i-1] + fib_series[i-2])
    return fib_series

fibonacci(20)
```I need to check the output to see the first 20 numbers in the Fibonacci series.
Action: Python_repl
Action Input: print(fibonacci(20))[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
I now have the first 20 numbers in the Fibonacci series.
Final Answer: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]

> Finished chain.


In [82]:
result["input"]

'Generate the first 20 numbers in the Fibonacci series'

In [83]:
result["output"]

'[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]'

In [87]:
question = "Can you tell me something about Langsmith?"

In [88]:
result = agent_executor.invoke({"input": prompt_template.format(q = question)})



> Entering new AgentExecutor chain...
I need to use the langsmith_search tool to find information about Langsmith.
Action: langsmith_search
Action Input: "Langsmith"Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though i

In [89]:
result["input"]

'"\nAnswer the following questions as best you can:\nQuestions: Can you tell me something about Langsmith?\n'

In [90]:
result["output"]

'Agent stopped due to iteration limit or time limit.'

In [91]:
question = "Tell something about Napoleon Bonaparte."

In [92]:
result = agent_executor.invoke({"input": prompt_template.format(q = question)})



> Entering new AgentExecutor chain...
I should use the Wikipedia tool to look up information about Napoleon Bonaparte.
Action: Wikipedia
Action Input: "Napoleon Bonaparte"Page: Napoleon
Summary: Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 182I have found the information I need about Napoleon Bonaparte.
Final Answer: Napoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution. He became Emperor of the French from 1804 to 1814 and again in 1815.

> Finished chain.


In [93]:
result["input"]

'"\nAnswer the following questions as best you can:\nQuestions: Tell something about Napoleon Bonaparte.\n'

In [94]:
result["output"]

'Napoleon Bonaparte was a French military and political leader who rose to prominence during the French Revolution. He became Emperor of the French from 1804 to 1814 and again in 1815.'

# 3- AGENTES: Tool calling agent

**CONSIDERACION 1:**

* `Tool calling` permite que un modelo detecte cuándo se deben llamar a una o más `tools` y responder con los inputs que se deben pasar.
* En una llamada a la API, se pueden describir las `tools` y hacer que el modelo genere un JSON que contenga los argumentos para utilizar dentro de cada `tool`.
* El objetivo de `Tool calling` es devolver las llamadas de las `tools` de manera más confiable que lo que se puede hacer usando una API genérica `text completion` o de `chat`.

**CONSIDERACION 2:**

* Ahora que tenemos un agente que puede generar JSONs estructurados, podemos vincular (`bind`) varias `tools` a un modelo y permitir que el modelo elija a cuál llamar.
* Se llamarán a diferentes `tools` hasta que el objetivo esté resuelto.

### 1- Iniciamos las `tools`

In [95]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

### 2- Creamos el prompt

In [96]:
from langchain import hub

In [97]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


### 3- Creamos el LLM

In [98]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 4- Creamos el agente

In [99]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [100]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [101]:
question = """Take 3 to the fifth power and multiply that by the sum of twelve and three,
then square the whole result
"""

results = agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.

> Finished chain.


In [102]:
results

{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three,\nthen square the whole result\n',
 'output': 'The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.'}

In [103]:
results["input"]

'Take 3 to the fifth power and multiply that by the sum of twelve and three,\nthen square the whole result\n'

In [104]:
results["output"]

'The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.'

# 4- AGENTES: SQL

**VENTAJAS:**

* Puede responder preguntas basadas en el `schema` de las bases de datos, así como en el `content` de las `db` como describir una tabla específica.
* Si obtiene un error, a través de la iteración detecta el error e intenta generar una respuesta correcta.
* Puede consultar la `db` tantas veces como sea necesario para responder la pregunta del usuario.
* Ahorrará `tokens` recuperando únicamente el esquema de las tablas relevantes.

**VAMOS A UTILIZAR:**

* El agente `create_sql_agent`
* Posee un `toolkit` llamado `SQLDatabaseToolkit` con herramientas para:
    - Crear y ejecutar queries.
    - Chequear la sintaxis de la query.
    - Dar descripciones de las tablas.

In [117]:
import sqlite3
from langchain_community.utilities import SQLDatabase

In [142]:
import sqlite3

sql_file_path = "/content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/Chinook_Sqlite.sql"
db_file_path = "/content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/chinook.db"


conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

with open(sql_file_path, 'r') as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()

conn.close()

print(f"Database created and saved to {db_file_path}")

Database created and saved to /content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/chinook.db


In [143]:
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

query = "SELECT * FROM Album LIMIT 10;"  # Ajusta esta consulta según tus necesidades
cursor.execute(query)

results = cursor.fetchall()

conn.close()

for row in results:
    print(row)

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)
(7, 'Facelift', 5)
(8, 'Warner 25 Anos', 6)
(9, 'Plays Metallica By Four Cellos', 7)
(10, 'Audioslave', 8)


In [149]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:///{db_file_path}")
print(db.dialect)
print(db.get_usable_table_names())
query = "SELECT * FROM Album LIMIT 10;"
results = db.run(query)
print("results:")
results

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
results:


"[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3), (6, 'Jagged Little Pill', 4), (7, 'Facelift', 5), (8, 'Warner 25 Anos', 6), (9, 'Plays Metallica By Four Cellos', 7), (10, 'Audioslave', 8)]"

In [150]:
from langchain_community.agent_toolkits import create_sql_agent

In [151]:
agent_executor = create_sql_agent(llm, db = db, agent_type = "openai-tools", verbose = True)

In [152]:
result = agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 

In [153]:
print(result["output"])

The total sales per country are as follows:

1. USA: $523.06
2. Canada: $303.96
3. France: $195.10
4. Brazil: $190.10
5. Germany: $156.48

Customers from the USA spent the most with a total of $523.06.


In [154]:
result = agent_executor.invoke("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The `PlaylistTrack` table has the following columns:
- PlaylistId (INTEGER, NOT NULL)
- TrackId (INTEGER, NOT NULL)

It has a composite primary key on columns PlaylistId and TrackId. Additionally, there are foreign key constraints referencing the PlaylistId column to the Playlist table's PlaylistId column, and the TrackId column to the Track table's TrackId column.

Here are 3 sample rows from th

In [155]:
print(result["output"])

The `PlaylistTrack` table has the following columns:
- PlaylistId (INTEGER, NOT NULL)
- TrackId (INTEGER, NOT NULL)

It has a composite primary key on columns PlaylistId and TrackId. Additionally, there are foreign key constraints referencing the PlaylistId column to the Playlist table's PlaylistId column, and the TrackId column to the Track table's TrackId column.

Here are 3 sample rows from the `PlaylistTrack` table:
1. PlaylistId: 1, TrackId: 3402
2. PlaylistId: 1, TrackId: 3389
3. PlaylistId: 1, TrackId: 3390


**Usando Few Shot como prompts dinámicos:**

In [156]:
examples = [
    {
        "input": "List all artists.",
        "query": "SELECT * FROM Artist;"
    },
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

**Creamos un selector para que seleccione los 5 ejemplos que más se parezcan a la consulta:**

In [157]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k = 5,
    input_keys = ["input"],
)

In [158]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables = ["input", "dialect", "top_k"],
    prefix = system_prefix,
    suffix = "",
)

In [159]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt = few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [160]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many artists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [161]:
agent = create_sql_agent(
    llm = llm,
    db = db,
    prompt = full_prompt,
    verbose = True,
    agent_type = "openai-tools",
)

In [162]:
result = agent.invoke({"input": "How many artists are there?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM Artist;'}`


[(275,)]There are 275 artists in the database.

> Finished chain.


In [163]:
result["output"]

'There are 275 artists in the database.'

# 5- AGENTES: Q&A RAG

In [105]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/drive/My Drive/ANALITICA_NOVIEMBRE/LANGCHAIN/state_of_the_union.txt")
documents = loader.load()

In [106]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [107]:
retriever = db.as_retriever()

In [108]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns excerpts from the 2022 State of the Union.",
)
tools = [tool]

In [109]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [110]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [111]:
result = agent_executor.invoke({"input": "hi, im bob"})

In [112]:
result["output"]

'Hello Bob! How can I assist you today?'

In [113]:
result = agent_executor.invoke(
    {
        "input": "what did the president say about ketanji brown jackson in the most recent state of the union?"
    }
)

In [114]:
result["output"]

"In the most recent State of the Union address, President Biden honored Justice Stephen Breyer and announced the nomination of Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court. He praised Judge Jackson as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. The President highlighted her background as a former top litigator, federal public defender, and consensus builder with broad support from various groups.\n\nAdditionally, President Biden shared a personal story about the impact of toxic burn pits on military personnel, mentioning his son Major Beau Biden's experience. He emphasized the need to address the health issues faced by veterans exposed to burn pits and announced initiatives to expand benefits for veterans suffering from respiratory cancers due to toxic exposures in Iraq and Afghanistan.\n\nFurthermore, the President addressed the importance of securing the border and reforming the immigration sy

In [115]:
result = agent_executor.invoke(
    {"input": "how long ago did the president nominate ketanji brown jackson?"}
)

In [116]:
result["output"]

'President Biden nominated Ketanji Brown Jackson four days ago.'